In [1]:
import os
os.getcwd()

'c:\\Desktop\\AgenticSQL\\code'

In [2]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [3]:
## download chinook database

## it is a small database with tables related to music, albums, track etc. so we can use it with sqlite.

import requests

url = "https://github.com/tharani001/MyDatasets/raw/refs/heads/main/db_samples/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    with open("../database/Chinook.db", "wb") as file:
        file.write(response.content)
    print("Chinook.db downloaded successfully.")
else:
    print(f"Failed to download Chinook.db. Status code: {response.status_code}")

Chinook.db downloaded successfully.
